In [ ]:
# importing necessary libraries
import itertools
import time
import keras
import numpy as np
import sklearn
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Dense, Dropout, Flatten



In [ ]:
def load_images_xy(file, crop=False):
    images_name = []
    images = []
    labels = []
    # looading in the text file with the image name and the label
    with open(file) as f:
        for line in f:
            # get the path of an image and the label
            image_dir, label = line.strip("\n").split(":")
            if label != '-1':
                if label =='1,2':
                    label = 3
                # read the image
                image = cv2.imread(f"{image_dir}")

                # resize to 224 x 224
                if crop:
                    image = cv2.resize(image, (224, 224))

                # from BGR to gray
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # append image and label to the list
                images_name.append(image_dir)
                images.append(image)
                labels.append(label)
        return images, labels, images_name


In [ ]:
images, labels, images_name = load_images_xy("XY_labels.txt")


In [ ]:
# normalise and transform to np array
def normalise_images(images, labels):
    # Convert to numpy arrays
    images = np.array(images, dtype=np.float32)
    labels = np.array(labels)
    labels = labels.astype(np.int)

    # Normalise the images
    images /= 255.0

    return images, labels


In [ ]:
images_norm, labels = normalise_images(images, labels)
indx = np.arange(0, len(labels))


In [ ]:
# shuffle the data
def shuffle_data(images_norm, labels, indx, images_name):
    X_data, y_data, indx, images_name = sklearn.utils.shuffle(
        images_norm, labels, indx, images_name, random_state=42)

    return X_data, y_data, indx, images_name


In [ ]:
X_data, y_data, indx, images_name = shuffle_data(
    images_norm, labels, indx, images_name)


In [ ]:
#Reshaping
X_data = X_data.reshape(-1, X_data.shape[1], X_data.shape[2], 1)

In [ ]:
# one hot encoder, e.g.: if it is a void then y=[0, 1]
y_data = to_categorical(y_data, num_classes=len(np.unique(y_data)))

In [ ]:
print(y_data.shape)

In [ ]:
# building light CNN model
xy_model_light = Sequential()
xy_model_light.add(Conv2D(2, kernel_size=(3, 3), activation='linear',
                    input_shape=(X_data.shape[1], X_data.shape[2], 1), padding='same'))
xy_model_light.add(LeakyReLU(alpha=0.1))
xy_model_light.add(MaxPooling2D((2, 2), padding='same'))
xy_model_light.add(Conv2D(4, (3, 3), activation='linear', padding='same'))
xy_model_light.add(LeakyReLU(alpha=0.1))
xy_model_light.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
xy_model_light.add(Conv2D(4, (3, 3), activation='linear',
                padding='same', name='just_do_it'))
xy_model_light.add(LeakyReLU(alpha=0.1))
xy_model_light.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
xy_model_light.add(Flatten())
xy_model_light.add(Dense(2, activation='linear'))
xy_model_light.add(LeakyReLU(alpha=0.1))
xy_model_light.add(Dense(y_data.shape[1], activation='softmax'))

xy_model_light.compile(loss=keras.losses.categorical_crossentropy,
                    optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])


In [ ]:
# saving untrained model for later
xy_model_light.save("xy_model_light_untrained.h5")


In [ ]:
# building heavy CNN model
xy_model_heavy = Sequential()
xy_model_heavy.add(Conv2D(8, kernel_size=(3, 3), activation='linear',
                          input_shape=(X_data.shape[1], X_data.shape[2], 1), padding='same'))
xy_model_heavy.add(LeakyReLU(alpha=0.1))
xy_model_heavy.add(MaxPooling2D((2, 2), padding='same'))
xy_model_heavy.add(Conv2D(16, (3, 3), activation='linear', padding='same'))
xy_model_heavy.add(LeakyReLU(alpha=0.1))
xy_model_heavy.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
xy_model_heavy.add(Conv2D(32, (3, 3), activation='linear',
                          padding='same', name="just_do_it"))
xy_model_heavy.add(LeakyReLU(alpha=0.1))
xy_model_heavy.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
xy_model_heavy.add(Flatten())
xy_model_heavy.add(Dense(16, activation='linear'))
xy_model_heavy.add(LeakyReLU(alpha=0.1))
xy_model_heavy.add(Dense(y_data.shape[1], activation='softmax'))

xy_model_heavy.compile(loss=keras.losses.categorical_crossentropy,
                       optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])


In [ ]:
# saving untrained model for later
xy_model_heavy.save("xy_model_heavy_untrained.h5")

In [ ]:
# initailizing to 30 and 20 epochs, need to chech performance vs number of epochs to decide on the epochs.
epochs_light = 30
epochs_heavy = 20
batch_size = 64


In [ ]:
# 80-20 train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42)


In [ ]:
history_heavy = xy_model_heavy.fit(X_train, y_train, epochs=epochs_heavy,
                                   batch_size=batch_size, validation_split=0.1)


In [ ]:
history_light = xy_model_light.fit(X_train, y_train, epochs=epochs_light,
                                   batch_size=batch_size, validation_split=0.1)


In [ ]:
def epoch_vs_performance_plot(history):
    # Plot the loss and accuracy curves for training and validation
    fig, ax = plt.subplots(2, 1)
    ax[0].plot(history.history['loss'], color='b', label="Training loss")
    ax[0].plot(history.history['val_loss'], color='r',
               label="validation loss", axes=ax[0])
    ax[1].set_xlabel("Number of epochs")
    legend = ax[0].legend(loc='best', shadow=True)

    ax[1].plot(history.history['accuracy'],
               color='b', label="Training accuracy")
    ax[1].plot(history.history['val_accuracy'],
               color='r', label="Validation accuracy")
    ax[1].set_xlabel("Number of epochs")
    legend = ax[1].legend(loc='best', shadow=True)


In [ ]:
def speed_and_acc(model, X_test, y_test):
    # shows the accuracy of model and number of images processed per second
    start_time = time.time()
    test_loss, test_acc = model.evaluate(X_test, y_test)
    delta_time = time.time() - start_time
    num_img = X_test.shape[0]
    print("--- %s images per second ---" % (num_img/delta_time))
    print(f"test acc: {test_acc}")


In [ ]:
epoch_vs_performance_plot(history_light)


In [ ]:
epoch_vs_performance_plot(history_heavy)


In [ ]:
# deciding on the epochs from the graph
epochs_light = 11


In [ ]:
# deciding on the epochs from the graph
epochs_heavy = 7

In [ ]:
xy_model_heavy = keras.models.load_model('xy_model_heavy_untrained.h5')


In [ ]:
xy_model_light = keras.models.load_model('xy_model_light_untrained.h5')


In [ ]:
# training on the new epochs number
history_light = xy_model_light.fit(X_train, y_train, epochs=epochs_light,
                                   batch_size=batch_size, validation_split=0.1)


In [ ]:
# training on the new epochs number
history_heavy = xy_model_heavy.fit(X_train, y_train, epochs=epochs_heavy,
                                   batch_size=batch_size, validation_split=0.1)


In [ ]:
# checking speed and accuracy
speed_and_acc(xy_model_heavy, X_test, y_test)

In [ ]:
# checking speed and accuracy
speed_and_acc(xy_model_light, X_test, y_test)

In [ ]:
# plotting confusion matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Predict the values from the train set with the light model
Y_pred = xy_model_light.predict(X_test)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis=1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=["No error", "Void"])


In [ ]:
# Predict the values from the train set with the heavy model
Y_pred = xy_model_heavy.predict(X_test)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis=1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=["No error", "Void"])


In [ ]:
# cross validation
# returns the list of accuracies
def cross_validation(model_name, X_data, y_data, epochs):
    acc = []
    kf = KFold(n_splits=5)
    #KFold with 5 splits
    for train_index, test_index in kf.split(X_data):
        X_train, X_test = X_data[train_index], X_data[test_index]
        y_train, y_test = y_data[train_index], y_data[test_index]

        # reinitailizing the model
        model = keras.models.load_model(model_name)

        history = model.fit(X_train, y_train, epochs=epochs,
                            batch_size=batch_size, validation_split=0.1)
        test_loss, test_acc = model.evaluate(X_test, y_test)
        print(f"test acc: {test_acc}")
        acc.append(test_acc)
        # break because im too lazy to wait so long for cross validation, later will run it fully
        # TODO: append test_acc to acc and make a new model with initial weights
    return np.array(acc)


In [ ]:
# doing cross-validation on heavy model
acc_heavy = cross_validation("xy_model_heavy_untrained.h5",
                       X_data, y_data, epochs_heavy)


In [ ]:
average_acc_heavy = np.average(acc_heavy)
st_dev_heavy = np.std(acc_heavy)


In [ ]:
print(f"Average accuracy of the heavy model: {average_acc_heavy}")
print(f"Standart deviation of the heavy model:{st_dev_heavy}")


In [ ]:
# doing cross-validation on ligth model
acc_light = cross_validation("xy_model_light_untrained.h5",
                             X_data, y_data, epochs_light)


In [ ]:
average_acc_light = np.average(acc_light)
st_dev_light = np.std(acc_light)


In [ ]:
print(f"Average accuracy of the light model: {average_acc_light}")
print(f"Standart deviation of the light model:{st_dev_light}")

In [ ]:
# loading untrained light model
xy_model_light = keras.models.load_model('xy_model_light_untrained.h5')


In [ ]:
# training light model on full dataset
xy_model_light.fit(X_data, y_data, epochs=epochs_light,
                                   batch_size=batch_size)


In [ ]:
xy_model_light.save("xy_model_light_FINAL.h5")


In [ ]:
# loading untrained heavy model
xy_model_heavy = keras.models.load_model('xy_model_heavy_untrained.h5')


In [ ]:
# training heavy model on the full dataset
xy_model_heavy.fit(X_data, y_data, epochs=epochs_heavy,
                   batch_size=batch_size)


In [ ]:
xy_model_heavy.save("xy_model_heavy_FINAL.h5")
